In [ ]:
import glob

readpath = 'E:/IP/dataset 256/HTC-1-M7/*.jpg'

labels = 'E:/IP/Labels/labels.csv'
objectClass = '1'

images = glob.glob(readpath)
labelfile = open(labels,'w')

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/iPhone-4s/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '2'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/iPhone-6/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '3'

images = glob.glob(readpath)
labelfile = open(labels,'a')

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/LG Nexus 5x/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '4'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Motorola Droid Maxx/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '5'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()

readpath = 'E:/IP/dataset 256/Motorola Nexus 6/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '6'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Motorola X/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '7'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Samsung Galaxy Note 4/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '8'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Samsung Galaxy S4/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '9'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Sony Nex 7/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '10'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()

In [ ]:
#for shuffle the image 
import random
# read the csv file 
labels = 'E:/IP/Labels/labels.csv'
#create a new csv file as shuffled_labels.csv where the image address will be shuffled
shuffled_labels = 'E:/IP/Labels/shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)
# create shuffle csv
shufflefile = open(shuffled_labels, "w") 
shufflefile.writelines(lines)
shufflefile.close()

In [ ]:
import numpy as np
import cv2
import pandas
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

labels = 'E:/IP/Labels/shuffled_labels.csv'
npzfile = 'E:/IP/Labels/labels.npz'
#create npz hiden  file 
df = pandas.read_csv(labels)

rows = df.iterrows()

X_temp = []
Y_temp = []

for row in rows:
    image = row[1][0]
    img = cv2.imread(image)
    #npimg = np.asarray(img)
    #npimg = npimg/255
    (b, g, r)=cv2.split(img)
    img=cv2.merge([r,g,b])
    imageClass = row[1][1]
    X_temp.append(img)
    Y_temp.append(imageClass)

       
# one hot encoding to represent one num as a array like 2= [0 1 0] in a 3 class cnn model     
encoder = LabelEncoder()
encoder.fit(Y_temp)
encoded_Y = encoder.transform(Y_temp)
Y = np_utils.to_categorical(encoded_Y)

np.savez(npzfile, X_train=X_temp,Y_train=Y)

In [ ]:
import os
from datetime import date, time, datetime
import numpy as np

sd = 7
np.random.seed(sd)
from tensorflow import set_random_seed
set_random_seed(sd)

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, ReduceLROnPlateau, EarlyStopping
from keras import initializers, regularizers
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.utils import plot_model
from keras.optimizers import Adam, Nadam, Adamax
from keras.constraints import max_norm
from keras import backend as K
K.set_image_data_format('channels_last')

save_dir = os.path.join(os.getcwd(),'saved_models_keras')
model_name = 'SPcup_trained_model.h5'


npzfile = 'E:/IP/Labels/labels.npz'

dataset =  np.load(npzfile)
x_train = dataset['X_train']
y_train = dataset['Y_train']

# function for writing the model.summary() to a text file.
def myprint(s):
    with open('\modelsummary\model_summary'+'_' + str(run_idx) + '.txt','w+') as f:
        print(s, file=f)

#x = float(x_train)/255.0
x /= np.max(x_train)

########### PARAMETERS LIST START############

#kernel_initializer = initializers.glorot_normal(seed=sd) can be tested instead of he_normal(seed=sd)
num_classes = 10
eps = 1e-6
dropout_rate_layers = 0.2
dropout_rate_dense = 0.05
maxpool_size = 2
kernel_size = 5
stride1 = 1
stride2 = 1
stride3 = 1
stride4 = 1

learning_rate = 0.001
lr_decay = 0.0

bias = False
val_split = 0.15 # Valid split = (NumOftestImageDataFromKaggle / totalNumOfImages)*100% = (2640/20k)*100 = 13.2%
batch_size = 32
nepoch = 20

l2reg_conv = 0.001 # for kernel_regularizer at conv layers
l2reg_dense = 0.005 # for kernel_regularizer at dense layers

maxnorm = 1000 #for kernel_constraint at conv layers
moment = 0.99 #for batchNormalization
run_idx = 11
pad_type = 'valid' # test using 'same'

# ReduceLROnPlateau params
reducelr_factor = 0.2
reducelr_patience = 3
cooldown = 0 #number of epochs to wait before resuming normal operation after lr has been reduced.

########### PARAMETERS LIST END ############

# CNN MODEL STARTS!

def CNN_model():
    
    model = Sequential()
    
    #layer 1
    model.add(Conv2D(32, kernel_size=kernel_size-1, 
                strides=stride,
                padding=pad_type,
                kernel_initializer=initializers.he_normal(seed=sd),
                data_format="channels_last", #(batch, height, width, channels)
                kernel_regularizer=regularizers.l2(l2reg_conv), #or kernel_regularizer=None.
                kernel_constraint=max_norm(maxnorm)
                use_bias=bias,
                input_shape=(256,256,3))
             )
    model.add(Activation('relu'))
    #batchNormalization AFTER relu is good => https://arxiv.org/abs/1511.06422
    model.add(BatchNormalization(epsilon=eps, axis=-1, momentum=moment))
    model.add(Dropout(rate=dropout_rate_layers, seed=sd))
    model.add(MaxPooling2D(pool_size=maxpool_size,
                         strides=maxpool_size,
                         data_format="channels_last"))
    
    #layer 2
    model.add(Conv2D(64,kernel_size=kernel_size,
                strides=stride,
                padding=pad_type,
                kernel_initializer=initializers.he_normal(seed=sd),
                data_format="channels_last", #(batch, height, width, channels)
                kernel_regularizer=regularizers.l2(l2reg_conv), #or kernel_regularizer=None.
                kernel_constraint=max_norm(maxnorm)
                use_bias=bias )
             )
    model.add(Activation('relu'))
#batchNormalization AFTER relu is good => https://arxiv.org/abs/1511.06422
    model.add(BatchNormalization(epsilon=eps, axis=-1, momentum=moment))
    model.add(Dropout(rate=dropout_rate_layers, seed=sd))
    model.add(MaxPooling2D(pool_size=maxpool_size,
                         # strides=maxpool_size,
                         data_format="channels_last"))
    
    
    #layer 3
    model.add(Conv2D(128,kernel_size=kernel_size, 
                strides=stride,
                padding=pad_type,
                kernel_initializer=initializers.he_normal(seed=sd),
                data_format="channels_last", #(batch, height, width, channels)
                kernel_regularizer=regularizers.l2(l2reg_conv), #or kernel_regularizer=None.
                kernel_constraint=max_norm(maxnorm)
                use_bias=bias )
             )
    model.add(Activation('relu'))
    #batchNormalization AFTER relu is good => https://arxiv.org/abs/1511.06422
    model.add(BatchNormalization(epsilon=eps, axis=-1, momentum=moment))
    model.add(Dropout(rate=dropout_rate_layers, seed=sd))
    model.add(MaxPooling2D(pool_size=maxpool_size,
                         # strides=maxpool_size,
                         data_format="channels_last"))
    
    #layer 4
    model.add(Conv2D(256,kernel_size=kernel_size, 
                strides=stride,
                padding=pad_type,
                kernel_initializer=initializers.he_normal(seed=sd),
                data_format="channels_last", #(batch, height, width, channels)
                kernel_regularizer=regularizers.l2(l2reg_conv), #or kernel_regularizer=None.
                kernel_constraint=max_norm(maxnorm)
                use_bias=bias)
             )
    model.add(Activation('relu'))
    #batchNormalization AFTER relu is good => https://arxiv.org/abs/1511.06422
    model.add(BatchNormalization(epsilon=eps, axis=-1, momentum=moment))
    model.add(Dropout(rate=dropout_rate_layers, seed=sd))
    model.add(MaxPooling2D(pool_size=maxpool_size,
                         # strides=maxpool_size,
                         data_format="channels_last"))
    
    model.add(Flatten())
    
    
    # now the fully connected layers!
    
    # DENSE layer 1
    model.add(Dense(256, activation='relu',
                   use_bias=bias,
                   kernel_initializer=initializers.he_normal(seed=sd),
                   kernel_regularizer=regularizers.l2(l2reg_dense),
                   kernel_constraint=max_norm(maxnorm)
                  ))
    #model.add(Dropout(rate=dropout_rate_dense, seed=sd))
    
    # DENSE layer 2
    model.add(Dense(num_classes, activation='softmax'))
    
    # types of OPTIMIZERS available
    adam = Adam(lr=learning_rate,decay=lr_decay)
    sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=moment, nesterov=True)
    rmsprop = RMSprop(lr=learning_rate)
    adamax = Adamax(lr=learning_rate)
    nadam = Nadam(lr=learning_rate)
    
    # types of LOSS FUNCTIONS available
    ctg_crossent = 'categorical_crossentropy'
    sparse_ctg_crossent = 'sparse_categorical_crossentropy'
    mse = 'mean_squared_error'
    
    model.compile(loss=catag_crossent , optimizer=adam , metrics=['accuracy'])
    

    return model

# CNN MODEL ENDS!

model = CNN_model()

model.summary(model.summary(print_fn=myprint))

# check will store the best model weight which will monitor the validation accuracy not train accuracy
file_path = 'best' + '_' + str(run_idx) + '.hdf5'

#saving weights as .hdf5 file. 
weights = model.get_weights()
print("Weights: ", weights)
model.save_weights(file_path) ####################### ????????????????

#setting up checkpoint save directory/ log names
checkpoint_path=os.path.join(os.path.join(os.getcwd(),'tmp'),str(date.today()))
if not os.path.isdir(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_name=checkpoint_path+'/'+str(run_idx)+'weights.{epoch:02d}-{val_acc:.4f}.hdf5'
log_name='logs'+ '_' + str(date.today()) + '_' + str(run_idx)

#checkpoints!
modelcheck=ModelCheckpoint(filepath=checkpoint_name,monitor='val_catagorical_acc', verbose=1,save_best_only=False,mode='max')
tensorbd=TensorBoard(log_dir='./logs/'+log_name,batch_size=batch_size,write_images=True)
csv_logger = CSVLogger('./logs/training_'+log_name+'.log',separator=',', append=True )
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reducelr_factor, patience=reducelr_patience, cooldown=cooldown, min_lr=0.0001, verbose=1, mode='min')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=6, mode='min')

checkpoints = [modelcheck, tensorbd, csv_logger, reduce_lr, early_stop]

#fitting model to for validation
training = model.fit(x, y_train, 
                     validation_split = val_split,
                     epochs=nepoch, 
                     batch_size=batch_size, 
                     verbose=2, #verbose=1 can result in slower training time.
                     shuffle=True,
                     callbacks = checkpoints)
print(training)

#plot_model() uses pyDot and GraphViz
plot_model(model, to_file='model.png')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
##### PLOTTING VAL_ACC AND lOSS USING MATPLOTLIB #####

import matplotlib.pyplot as plt
%matplotlib inline

# show the loss and accuracy
loss = training.history['loss']
val_loss = training.history['val_loss']
acc = training.history['acc']
val_acc = training.history['val_acc']

# loss plot
tra = plt.plot(loss)
val = plt.plot(val_loss, 'r')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend(["Training", "Validation"])

plt.show()

# accuracy plot
plt.plot(acc)
plt.plot(val_acc, 'r')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Accuracy')
plt.legend(['Training', 'Validation'], loc=4)
plt.show()

In [ ]:
# SOHEL'S FIRST CODE FOR TESTING.

#predict the class
y = model.predict_classes(X)
# y is a one dim array we have to convert it to a classs num like 3,4 
classno = np.ndarray.tolist(y)

dict = {0: 'LG', 1:'moto maxx', 2: 'sony,}
objectClass = dict[classno[0]]
print(objectClass)
print(y)
        
#yPred = model.predict_classes(x_valid,batch_size=32,verbose=1)

#np.savetxt('mnist_output.csv', np.c_[range(1,len(yPred)+1),yPred], delimiter=',', header = 'ImageId,Label', comments = '', fmt='%d')

#font = cv2.FONT_HERSHEY_SIMPLEX
#cv2.putText(img, objectClass,(50,50), font, 2, (200,255,0), 5, cv2.LINE_AA)
#cv2.imshow('Prediction',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
# GOT FROM THIS LINK: https://goo.gl/kRPp9p

# load the best model
model = load_model(file_path)

# load test_data
test_data = pd.read_csv('../input/test.csv')

# prediction
pred = model.predict(test_features, batch_size=16, 
                       verbose=1)

# convert predicions from categorical back to 0...9 digits
pred_digits = np.argmax(pred, axis=1)

submission = pandas.DataFrame({'Label': pred_digits})
submission.index += 1
submission.index.name = "ImageId"

submission.to_csv('submission.csv')

In [ ]:
# GOT FROM THIS LINK: https://goo.gl/wk2WwP

############## LOAD MODEL ########################
net = load_model(file_path)

############## PREDICT ########################

test_imgs = os.listdir(test_image_path)
predict_df = pd.DataFrame(index=np.arange(0, nb_rows), columns=classes)

count = 0
for img in test_imgs:
    img = image.load_img(test+image_path+img, target_size=(256, 256))
    x = image.img_to_array(img)/255.0
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    #Actual prediction
    preds = net.predict_proba(x)[0]
    predict_df.loc[count] = preds
    count += 1
    
img_name_df = pd.read_csv("../submission/sample_submission_stg1.csv")
img_name_df = img_name_df[nm_img]

predict_df.insert(0,nm_img,img_name_df)

predict_df.to_csv('nature_predicts.csv',index=False)

In [ ]:
#KAGGLE SUBMISSION TYPE.

############## generating X_test ##############
import os

list_paths = []
for subdir, dirs, files in os.walk("E:\IP"):
    for file in files:
        #print(os.path.join(subdir, file))
        filepath = subdir + os.sep + file
        list_paths.append(filepath)
        
list_test = [filepath for filepath in list_paths if "test/" in filepath]
index = [os.path.basename(filepath) for filepath in list_test]

X_test = np.array([read_and_resize(filepath) for filepath in list_test])

############# Generation complete ################

file_path = 'best.hdf5'
model.load_weights(file_path)

predicts = model.predict(X_test)
predicts = np.argmax(predicts, axis=1)
predicts = [label_index[p] for p in predicts]

df = pd.DataFrame(columns=['fname', 'camera'])
df['fname'] = index
df['camera'] = predicts
df.to_csv("sub.csv", index=False) 